**INSTALL LIBRARIES**

In [18]:
! pip install polars
! pip install requests


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


**DATABASE CONNECTION SETUP**

In [19]:
import psycopg2
from psycopg2.extras import execute_values

db_config = {
    'dbname': 'mydatabase',
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 5432
}

connection = psycopg2.connect(**db_config)
print("Connected!")


Connected!


**CREATE TABLE AND ADD DATA WITH ERROR HANDLING**

In [20]:
try:
    cursor = connection.cursor()

    create_table_query = """
    CREATE TABLE IF NOT EXISTS stock_data (
        id SERIAL PRIMARY KEY,
        symbol VARCHAR(10) NOT NULL,
        timestamp TIMESTAMP(0) WITHOUT TIME ZONE NOT NULL,
        open DECIMAL(10,4),
        high DECIMAL(10,4),
        low DECIMAL(10,4),
        close DECIMAL(10,4),
        volume BIGINT,
        UNIQUE(symbol, timestamp)
    );
    """
    cursor.execute(create_table_query)
    connection.commit()
    print("Table ensured!")

except Exception as e:
    print("Error during table creation:", e)


Table ensured!


**REQUEST FROM APLHAVANTAGE API WITH ERROR HANDLING**

In [21]:
import requests
from datetime import datetime

API_KEY = "42JIJGRH51L2GBTP"
SYMBOL = "IBM"
INTERVAL = "5min"
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={SYMBOL}&interval={INTERVAL}&apikey={API_KEY}'

r = requests.get(url)
data = r.json()

time_series_key = f"Time Series ({INTERVAL})"
if time_series_key not in data:
    print("Error: Invalid API response or rate limit exceeded.")
    exit()

time_series = data[time_series_key]

records = []
for timestamp, values in time_series.items():
    dt_obj = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    formatted_timestamp = dt_obj.strftime("%Y-%m-%d %H:%M")

    records.append({
        "symbol": SYMBOL,
        "timestamp": formatted_timestamp,
        "open": float(values["1. open"]),
        "high": float(values["2. high"]),
        "low": float(values["3. low"]),
        "close": float(values["4. close"]),
        "volume": int(values["5. volume"])
    })

print(records[:5])


[{'symbol': 'IBM', 'timestamp': '2025-01-28 19:55', 'open': 225.5, 'high': 225.5, 'low': 225.5, 'close': 225.5, 'volume': 20}, {'symbol': 'IBM', 'timestamp': '2025-01-28 19:50', 'open': 225.55, 'high': 225.55, 'low': 225.55, 'close': 225.55, 'volume': 17}, {'symbol': 'IBM', 'timestamp': '2025-01-28 19:45', 'open': 225.3562, 'high': 225.55, 'low': 225.3501, 'close': 225.55, 'volume': 11}, {'symbol': 'IBM', 'timestamp': '2025-01-28 19:40', 'open': 225.54, 'high': 225.55, 'low': 225.35, 'close': 225.55, 'volume': 65}, {'symbol': 'IBM', 'timestamp': '2025-01-28 19:35', 'open': 225.54, 'high': 225.54, 'low': 225.54, 'close': 225.54, 'volume': 1}]


**INSERT DATA INTO TABLE WITH ERROR HANDLING**

In [22]:
try:
    insert_query = """
    INSERT INTO stock_data (symbol, timestamp, open, high, low, close, volume)
    VALUES %s
    ON CONFLICT (symbol, timestamp) DO NOTHING;
    """
    
    values = [
        (row["symbol"], row["timestamp"], row["open"], row["high"], row["low"], row["close"], row["volume"])
        for row in records
    ]
    
    execute_values(cursor, insert_query, values)

    connection.commit()
    print("Data inserted successfully!")

except Exception as e:
    print("Error inserting data:", e)

finally:
    cursor.close()
    connection.close()


Data inserted successfully!
